In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/08 19:06:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/08 19:06:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/08 19:06:30 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
df_green = spark.read.parquet('download_data/data/pq/green/*/*')

In [4]:
df_green.createOrReplaceTempView ('green')

In [5]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [6]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-28 19:00:00| 134|193.61000000000007|            17|
|2020-01-22 19:00:00|  65| 657.0300000000001|            41|
|2020-01-27 08:00:00|  17|             85.56|             4|
|2020-01-02 09:00:00|  66|229.39999999999998|            12|
|2020-01-02 12:00:00|  89|310.28000000000003|            14|
|2020-01-07 12:00:00|  66|             179.5|             9|
|2020-01-03 08:00:00| 223|165.90000000000003|             9|
|2020-01-17 10:00:00|  41| 638.2699999999999|            49|
|2020-01-02 11:00:00|  26|             198.6|             7|
|2020-01-12 20:00:00| 247|36.900000000000006|             3|
|2020-01-12 15:00:00|  10|             75.92|             1|
|2020-01-24 04:00:00| 129|148.89000000000001|            13|
|2020-01-04 20:00:00|  25| 369.5700000000001|            23|
|2020-01-22 10:00:00|  7

In [7]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('download_data/data/report/revenue/green', mode='overwrite')

In [14]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [15]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('download_data/data/report/revenue/green', mode='overwrite')

In [16]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")

In [17]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('download_data/data/report/revenue/green', mode='overwrite')

In [18]:
df_yellow = spark.read.parquet('download_data/data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [19]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [22]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('download_data/data/report/revenue/yellow', mode='overwrite')

In [31]:
df_green_revenue = spark.read.parquet('download_data/data/report/revenue/green')
df_green_revenue.show()
df_yellow_revenue = spark.read.parquet('download_data/data/report/revenue/yellow')
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-02-02 01:00:00| 264|             32.46|             3|
|2020-01-18 11:00:00| 223|             86.38|             7|
|2020-01-15 09:00:00| 126|             58.22|             2|
|2020-01-03 10:00:00| 167|114.42999999999999|             6|
|2020-02-15 11:00:00|  29|             20.74|             1|
|2020-01-05 09:00:00|  82|104.27999999999999|            10|
|2020-02-13 07:00:00|   7|            152.18|            11|
|2020-01-19 22:00:00|  74| 609.5299999999999|            48|
|2020-02-11 05:00:00| 102|             35.04|             1|
|2020-01-18 21:00:00| 259|              36.0|             1|
|2020-02-07 10:00:00|  75| 969.5999999999993|            64|
|2020-01-16 23:00:00|  29|              61.8|             1|
|2020-02-25 06:00:00| 146|             16.05|             1|
|2020-02-16 04:00:00|  5

In [32]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [33]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [34]:
df_join.write.parquet('download_data/data/report/revenue/total', mode='overwrite')

In [28]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [30]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   3|              null|                null|              25.0|                    1|
|2020-01-01 00:00:00|   4|              null|                null|1004.3000000000002|                   57|
|2020-01-01 00:00:00|   7| 769.7299999999996|                  45| 455.1700000000001|                   38|
|2020-01-01 00:00:00|  12|              null|                null|             107.0|                    6|
|2020-01-01 00:00:00|  37|            175.67|                   6|161.60999999999999|                    7|
|2020-01-01 00:00:00|  40|168.97999999999996|                   8|             89.97|                    5|
|2020-01-01 00:00:00|  45|  

In [35]:
df_zones = spark.read.parquet('zones/')

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/home/datatalks_jan/Data_Eden/8_pySpark_pilot/zones.